[SOURCE 32](https://www.youtube.com/watch?v=g3OZJ6skE_U)

In [ ]:
# Check directory
import os
cd = os.getcwd()
os.listdir(cd)

In [1]:
# Import necessary library
import cv2 as cv

# Step 1 - Read image and set pixel size
img = cv.imread("thinsection_image2.jpg")
pixels_to_um = 0.5 # 1 pixel = 0.5 um or 500 nm
cv.imshow("Original", img)
# convert image into grayscale
img_g1 = cv.imread("thinsection_image2.jpg", 0)
# or..
# img_g = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
cv.imshow("Gray", img_g1)

# step 2
from matplotlib import pyplot as plt
# plt.hist(img.flat, bins = 100, range=(0,255))
ret, thresh = cv.threshold(img_g1, 0, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)
cv.imshow("Threshold", thresh)

# Step 3
import numpy as np
kernel = np.ones((3,3), np.uint8)
eroded = cv.erode(thresh, kernel, iterations=1)
dilated = cv.dilate(eroded, kernel, iterations=1)
# cv.imshow("Eroded", eroded)
# cv.imshow("Dilated", dilated)

from scipy import ndimage
from skimage import io, color, measure
mask = dilated == 255
# io.imshow(mask[250:280, 250:280])


cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
# Check image size
height, width, channels = img.shape
print(height, width, channels)

In [ ]:
# Step 4
s = [[1,1,1],[1,1,1],[1,1,1]]
labeled_mask, num_labels = ndimage.label(mask, structure=s)

img2 = color.label2rgb(labeled_mask, bg_label=0)
cv.imshow("Colored_label", img2)

cv.waitKey(0)
cv.destroyAllWindows()


In [ ]:
# Step 5
clusters = measure.regionprops(labeled_mask, img)

for prop in clusters:
    print('Label: {} Area: {}'.format(prop.label, prop.area))

In [ ]:
propList = ['Area', 
            'equivalent_diameter',
            'orientation',
            'MajorAxisLength',
            'MinorAxisLength',
            'Perimeter',
            'MinIntensity',
            'MeanIntensity',
            'MaxIntensity']

In [ ]:
output_file = open('image_measurements.csv', 'w')
output_file.write((',' + ",".join(propList) + '\n'))

In [ ]:
for cluster_props in clusters:
    output_file.write(str(cluster_props['Label']))
    for i, prop in enumerate(propList):
        if(prop == 'Area'):
            to_print = cluster_props[prop]*pixels_to_um**2
        elif(prop == 'orientation'):
            to_print = cluster_props[prop]*57.2958
        elif(prop.find('Intensity') < 0):
            to_print = cluster_props[prop]*pixels_to_um
        else:
            to_print = cluster_props[prop]
            
        output_file.write(',' + str(to_print))
    output_file.write('\n')